## Text to MySQL Agent

- This simple agent responds to a text message with a MySQL query execution result.
- The agent is built using LangGraph
- We will start with simple linear flow and then add more complex flows
- For this example, We will use `Chinook` database which is sample database available for `sqlite`
- You can tryout this for any database `sqlite` or `mysql` or `postgresql` by changing the connection string

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')

In [ ]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("Chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


In [ ]:
from langchain_ollama import ChatOllama

# model = "llama3.2:3b"
model = "qwen2.5"
llm = ChatOllama(model=model, base_url="http://localhost:11434")
print(llm.invoke("Hello, how are you?"))


from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
# Application state
from typing import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


# Convert question to SQL query
# We will pull a prompt from the Prompt Hub to instruct the model.
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

# assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

# query_prompt_template



In [ ]:
from typing import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    print("Result", result)
    return {"query": result["query"]}


# query output metadata
QueryOutput.__annotations__

In [ ]:
llm.with_structured_output(QueryOutput)

write_query({"question": "How many Employees are there?"})

In [ ]:
# Execute query
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDataBaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

execute_query({'query': 'SELECT COUNT(*) FROM Employee'})

In [ ]:
# Generate answer
# Finally, our last step generates an answer to the question 
# given the information pulled from the database:

def generate_answer(state: State):
    """Answer question using retrieved information as context."""
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}


question = "How many Employees are there?"
query = write_query({"question": question})
result = execute_query(query)



In [ ]:
state = {"question": question, **query, **result}
print(state)

generate_answer(state)

In [ ]:
# Orchestrating with LangGraph
# Finally, we compile our application into a single graph object.
# In this case, we are just connecting the three steps into a single sequence.

from langgraph.graph import START, StateGraph
from IPython.display import Image, display

graph_builder = StateGraph(State)

graph_builder.add_node("write_query", write_query)
graph_builder.add_node("execute_query", execute_query)
graph_builder.add_node("generate_answer", generate_answer)

graph_builder.add_edge(START, "write_query")
graph_builder.add_edge("write_query", "execute_query")
graph_builder.add_edge("execute_query", "generate_answer")


graph = graph_builder.compile()


display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
query = {"question": "How many employees are there?"}

for step in graph.stream(query, stream_mode="updates"):
    print(step)

### AGENTS

- They can query the database as many times as needed to answer the user question.
- They can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- They can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table).

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].pretty_print()

In [ ]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [ ]:
# Initializing agent
# We will use a prebuilt LangGraph agent to build our agent

from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm,
                                    tools,
                                    state_modifier=system_message)

display(Image(agent_executor.get_graph().draw_mermaid_png()))

In [ ]:
question = "Which country's customers spent the most?"
query = {"messages": [HumanMessage(question)]}
for step in agent_executor.stream(query, stream_mode="values"):
    step["messages"][-1].pretty_print()